In [1]:
# In this notebook I take all the articles that I crawled from psychologytoday.com and perfom some explorative analysis.
# I finally seperate out all the articles with the category 'relationships' for further analysis.
# This notebook was create with Databricks Community Edition: community.cloud.databricks.com so it will not work locally.

In [2]:
# Imports
import pandas as pd
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import monotonically_increasing_id 

In [3]:
# The dataset is not very big so we can read in everything as pandas dataframes
data_pd_df = pd.concat([pd.read_pickle(f'/dbfs/FileStore/tables/{FileInfo.name}') for FileInfo in dbutils.fs.ls('/FileStore/tables/') if FileInfo.name.startswith('PT_data')])

# There are some duplicte articles that have either been posted several times, or have been crawled
# more than once. In order to identify them as duplicates we drop time_crawled
data_pd_df = data_pd_df.drop(['time_crawled', 'crawled_from'], axis=1)

schema = StructType([ StructField("date", DateType(), True)\
                       ,StructField("category", StringType(), True)\
                       ,StructField("title", StringType(), True)\
                       ,StructField("author", StringType(), True)\
                       ,StructField("blog_post_link", StringType(), True)\
                       ,StructField("teaser_text", StringType(), True)\
                       ,StructField("article_div", StringType(), True)])

df = spark.createDataFrame(data_pd_df, schema=schema).distinct()

In [4]:
# Take a look at the data
display(df.limit(10))

date category title author blog_post_link teaser_text article_div 2017-03-29 Stress Ten Simple Tips to Manage the Stress of Moving Barbara Markway Ph.D. /intl/blog/living-the-questions/201703/ten-simple-tips-manage-the-stress-moving Is moving stressing you out? A few tweaks can go a long way. [ , , , , , , , , , , , 
 
 
 
 Source: Can Stock Photo 
 
 
 
 My husband and I have moved plenty of times, but it’s always been local moves where we could haul stuff in our cars and maybe rent a U-Haul for the big stuff. This time we’re moving to another city due to a job transfer, and it’s a whole new ballgame. Throw in the fact that we’re 10 years older than our last move, and well, it’s not been as easy as I remember it. We’re in the home stretch, so let me be honest and say we’ve broken almost all of the tips below, so I’m offering these in retrospect. 
 
 1. Start earlier than you think you need to. If you have the luxury of knowing about a move well in advance, double the time you think you’ll need to get everything done. I don’t know why, but it seems like your stuff multiplies in volume once you start packing.   
 2. Leave a little “sanctuary” area in your house or apartment that doesn’t get packed until the last minute. I am a meditator and did not realize how much the cues of my chair and my inspirational books facilitate my regular practice. Once my chair was gone and my books were packed up, I simply stopped meditating. This was not good for my stress level. 
 
 3. Similar to tip two, leave out a few things you use for self-care activities . I like to do watercolor painting and of course, all my painting supplies are packed. 
 4. Make time for regular exercise. We still belong to our local gym, but have we gone? No. We’re too busy and tired from packing. My husband has been faithful about running on the treadmill in the basement, but aside from walking my dogs, I’ve let exercise go by the wayside. 
 
 5. Expect to be crabby. Moving is stressful . You’re going to be cranky at times. Don’t make things worse by beating yourself up because you’re not in a good mood all the time. On the other hand, don’t take your bad mood out on other people. 
 6. Accept help. I should have read this: 7 Effective Ways to Ask for Help (and Get It) . 
 
 7. Make a list of all the fun things you’re going to do in your new city. This will keep you motivated during the hard work. And don’t lose the list! 
 8. Keep all important documents in a special place. Again, don’t forget where this special place is. 
 9. Leave time for goodbyes. Suddenly people I see only every six months want to get together. It’s so nice that they’re thinking of me, but now I feel like I don’t have enough time, let alone the emotional energy, to be with all these people. Just as with tip #1, I should’ve started this part much sooner. 
 
 10. Practice self-compassion . Realize that this is a hard time. You’re going through a lot of emotional and existential changes. Know that you’re doing the best you can. Know that you’re not alone. People have moved plenty of times and gone through the same feelings as you. Be kind to yourself.  
 Bonus Tip: Laugh! Here are 21 Quotes about Laughter 
 
 We don’t plan to move again any time soon, but if we do, I hope I remember this list. 
 You might also like: 10 Essential Skills for Couples Coping with Stress 
 You can follow me on Facebook and Twitter. 
 , , , , , , , , , , , , , ] 2017-03-24 Law and Crime Explanation or Excuses for Stealing? Stanton E. Samenow Ph.D. /intl/blog/inside-the-criminal-mind/201703/explanation-or-excuses-stealing The criminal evaluates his evaluator. [ , , , , , , , , , , , When offenders are held accountable, unless they make a full confession, they offer explanations for why they behaved as they did. Their after-the-fact statements are usually intended to gain sympathy and minimize culpability. They focus on people and circumstances outside themselves. Often their statements sound plausible even to seasoned cri

In [5]:
# How many articles are there?
df.count()

Out[23]: 39909

In [6]:
# Regsiter Dataframe as temporary view
df.createOrReplaceTempView("pt_data")

In [7]:
# What categories are there?
display(spark.sql('SELECT DISTINCT(category) FROM pt_data'))

category
Anger
Education
Philosophy
Self-Harm
Post-Traumatic Stress Disorder
Dementia
Bullying
Postpartum Depression
Behavioral Economics
Sociopathy


In [8]:
# How many categories are there?
display(spark.sql('SELECT COUNT(DISTINCT(category)) as distinct_categories FROM pt_data'))

distinct_categories
141


In [9]:
# What are the number of articles for each cetegory?
display(spark.sql('Select category, count(title) as count FROM pt_data GROUP BY category ORDER BY count(title) DESC'))

category,count
Relationships,4418
Parenting,1918
Self-Help,1890
Happiness,1759
Work,1518
Health,1301
Anxiety,1272
Sex,1114
Animal Behavior,1046
Politics,1000


In [10]:
# Display the number of articles for every category
display(spark.sql('Select category, count(title)as count FROM pt_data GROUP BY category ORDER BY count(title) DESC'))

category,count
Relationships,4418
Parenting,1918
Self-Help,1890
Happiness,1759
Work,1518
Health,1301
Anxiety,1272
Sex,1114
Animal Behavior,1046
Politics,1000


In [11]:
# How many articles were posted per day?
display(spark.sql('SELECT date, count(title) FROM pt_data GROUP BY date ORDER BY date ASC'))

date,count(title)
2016-01-01,21
2016-01-02,14
2016-01-03,20
2016-01-04,25
2016-01-05,28
2016-01-06,18
2016-01-07,25
2016-01-08,27
2016-01-09,11
2016-01-10,20


In [12]:
# Was there a certain topic on the 2018-08-27 that was written about, that accounts for the spike in publiched article count?
display(spark.sql('SELECT date, category, title, author, teaser_text FROM pt_data WHERE date = "2018-08-27"'))

date,category,title,author,teaser_text
2018-08-27,Happiness,It’s Not Just Business As Usual,Michael F. Kay,"The cost of bad, incomplete, or improper advice is too high."
2018-08-27,Self-Help,Are You Sure You Know What You’re Feeling?,Guy Winch Ph.D.,Knowing which emotions you experience is not as easy as you might think.
2018-08-27,Depression,Perfectionism Can Drive Success and Unhappiness,Leslie Becker-Phelps Ph.D.,"If you have the sense that you are only as good as your last accomplishment, your drive to be perfect is pulling you down. Learn how to set high standards AND be happy."
2018-08-27,Work,Why Work Matters: 10 Reasons to Celebrate Labor Day,Gina Barreca Ph.D.,My friend Heidi gave birth to her daughter on Labor Day. She tends to take things literally.
2018-08-27,Relationships,Top 10 Tips To Reduce Relationship Baggage,Barton Goldsmith Ph.D.,We all come into our relationships with items from our past which have an affect on our relationship. Some of these things make us better partners and some do not.
2018-08-27,Relationships,Top 10 Tips To Reduce Relationship Baggage,Barton Goldsmith Ph.D.,We all come into our relationships with items from our past which have an affect on our relationship. Some of these things make us better partners and some do not.
2018-08-27,Eating Disorders,Do Muscles Make the Man?,Karen L. Blair Ph.D.,Gay and bisexual men are not exempt from concerns about appearance but is there a unique motivator behind some sexual minority men's drive for muscularity and masculinity?
2018-08-27,Work,Salvation: Time to Say Good-Bye to My Past,Shireen Jeejeebhoy,Brain injury defaults alertness to low. Dreams and work die in broken neurons. But my mind wasn’t bopping like some sort of superball out of control. Brain biofeedback was working!
2018-08-27,Ethics and Morality,When Truth Isn’t Truth,Gleb Tsipursky Ph.D.,See how Rudy Giuliani manipulates us using psychological tactics from climate change deniers and tobacco lobbyists
2018-08-27,Relationships,Strengthening Interracial Relationships,"Holly Parker, Ph.D.",How can interracial couples bolster one another and their bond from within as they encounter resistance and unjust treatment from without?


In [13]:
# What is the median and average count of articles posted per day?
median = np.median(np.array(spark.sql('SELECT count(title) FROM pt_data GROUP BY date').select('count(title)').rdd.flatMap(lambda x: x).collect()))
print(f'Median posts published per day: {median}')

avg = np.average(np.array(spark.sql('SELECT count(title) FROM pt_data GROUP BY date').select('count(title)').rdd.flatMap(lambda x: x).collect()))
print(f'Average posts published per day: {avg}')

Median posts published per day: 25.0
Average posts published per day: 25.831067961165047

In [14]:
# Get the published articles per day for the top ten popular categories.
top_ten_pub = spark.sql('SELECT date, category, COUNT(title) FROM pt_data WHERE category in (Select category FROM pt_data GROUP BY category ORDER BY count(title) DESC LIMIT 10) GROUP BY date, category')

In [15]:
pd_top_ten_pub = top_ten_pub.toPandas()

In [16]:
# To not miss a day we need to create a date range to later join the dataframe on 
df_all_dates = pd.date_range(start=pd_top_ten_pub.date.min(), end=pd_top_ten_pub.date.max()).to_frame(index=True, name='date')

In [17]:
def display_category(category_name, df_all_dates=df_all_dates, pd_top_ten_pub=pd_top_ten_pub):
  
  temp_df = pd.merge(
        df_all_dates, 
        pd_top_ten_pub[pd_top_ten_pub['category'] == category_name].astype({'date': 'datetime64'}), 
        on='date', how='left').drop(['category'], axis=1).fillna(0)
  temp_df = temp_df.rename(columns={"count(title)": "count"})
  temp_df = spark.createDataFrame(temp_df)
  temp_df = temp_df.withColumn('date', temp_df['date'].cast('date'))
  return temp_df

In [18]:
display(display_category("Relationships"))

date,count
2016-01-01,3.0
2016-01-02,0.0
2016-01-03,4.0
2016-01-04,4.0
2016-01-05,2.0
2016-01-06,0.0
2016-01-07,4.0
2016-01-08,3.0
2016-01-09,1.0
2016-01-10,2.0


In [19]:
display(display_category("Parenting"))

date,count
2016-01-01,2.0
2016-01-02,0.0
2016-01-03,0.0
2016-01-04,0.0
2016-01-05,0.0
2016-01-06,1.0
2016-01-07,1.0
2016-01-08,0.0
2016-01-09,0.0
2016-01-10,0.0


In [20]:
display(display_category("Anxiety"))

date,count
2016-01-01,1.0
2016-01-02,0.0
2016-01-03,0.0
2016-01-04,0.0
2016-01-05,0.0
2016-01-06,0.0
2016-01-07,0.0
2016-01-08,0.0
2016-01-09,0.0
2016-01-10,0.0


In [21]:
display(display_category("Self-Help"))

date,count
2016-01-01,0.0
2016-01-02,1.0
2016-01-03,2.0
2016-01-04,0.0
2016-01-05,1.0
2016-01-06,7.0
2016-01-07,0.0
2016-01-08,2.0
2016-01-09,0.0
2016-01-10,0.0


In [22]:
display(display_category("Happiness"))

date,count
2016-01-01,1.0
2016-01-02,1.0
2016-01-03,1.0
2016-01-04,0.0
2016-01-05,2.0
2016-01-06,0.0
2016-01-07,0.0
2016-01-08,1.0
2016-01-09,1.0
2016-01-10,1.0


In [23]:
# How many different authors are there
display(spark.sql('SELECT COUNT(DISTINCT(author)) as number_of_authors FROM pt_data'))

number_of_authors
2301


In [24]:
# Some articles are posted multiple times
display(spark.sql('SELECT title, count(title) FROM pt_data WHERE category = "Relationships" GROUP BY title ORDER BY count(title) DESC'))

title,count(title)
Great Communication,3
6 Troubling Signs of Psychological Abuse,2
Timing Matters When It Comes to Relationship Success,2
Estranged? Is It Time to Reach Out and Begin Healing?,2
Can We Prevent Adolescent Sexual Violence?,2
Are You a Prisoner of Perfection?,2
"Together, Against, Apart",2
Why Your Couples’ Therapy Didn’t Last,2
Abusive Teen Dating Relationships,2
Who Do You Trust?,2


In [25]:
# How many articles where postet multiple times?
spark.sql('SELECT title FROM (SELECT title, count(title) as c FROM pt_data WHERE category = "Relationships" GROUP BY title) WHERE c > 1').count()

Out[53]: 97

In [26]:
# I want to analyse the 'Relationships' posts locally so I seperate the data and save it to a pickle file
only_relationships = spark.sql('SELECT * FROM pt_data WHERE category = "Relationships"').toPandas()
only_relationships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4418 entries, 0 to 4417
Data columns (total 7 columns):
date 4418 non-null object
category 4418 non-null object
title 4418 non-null object
author 4418 non-null object
blog_post_link 4418 non-null object
teaser_text 4418 non-null object
article_div 4418 non-null object
dtypes: object(7)
memory usage: 241.7+ KB

In [27]:
only_relationships.to_pickle('/dbfs/FileStore/tables/relationship_posts.pkl')